In [10]:
import pandas as pd

# Load data
workers = pd.read_csv('workers.csv')
job_locations = pd.read_csv('job_locations.csv')

# Example preprocessing steps
# Convert categorical variables to numerical ones (e.g., skill set, language proficiency)
workers['skill_set'] = workers['skill_set'].astype('category').cat.codes
workers['language_proficiency'] = workers['language_proficiency'].astype('category').cat.codes
job_locations['required_skill_set'] = job_locations['required_skill_set'].astype('category').cat.codes
# # Calculate proximity (assuming lat/lon data for simplicity)
# def calculate_distance(lat1, lon1, lat2, lon2):
#     from geopy.distance import geodesic
#     return geodesic((lat1, lon1), (lat2, lon2)).km

# job_locations['proximity'] = job_locations.apply(
#     lambda row: calculate_distance(row['lat'], row['lon'], workers['lat'], workers['lon']), axis=1
# )


In [11]:
# Create features for matching
features = pd.DataFrame()
features['worker_id'] = workers['worker_id']
features['job_location_id'] = job_locations['job_location_id']
features['skill_match'] = (workers['skill_set'] == job_locations['required_skill_set']).astype(int)
# features['proximity'] = job_locations['proximity']
features['cost'] = workers['wage_rate']
print(features)

# You can add more features as necessary


   worker_id  job_location_id  skill_match  cost
0          1              101         True    25
1          2              102         True    30
2          3              103         True    28
3          4              104         True    35
4          5              105         True    26


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assume you have a label indicating successful matches
# features['label'] = ... # Your label data here

# Split data
X = features.drop('skill_match', axis=1)
y = features['skill_match']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 1.0
